In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
from collections import defaultdict
import itertools
import os
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from IPython.display import Markdown
from matminer.datasets import load_dataset
from pymatgen.core import Composition

from modnet.preprocessing import MODData
#from modnet.models import MODNetModel
from modnet.featurizers import MODFeaturizer
from modnet.featurizers.presets import DeBreuck2020Featurizer
from modnet.models import MODNetModel

from gptchem.data import get_matbench_glass
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
import modnet
modnet.__version__

'0.1.12'

In [4]:
df =  get_matbench_glass()

In [5]:
class CompositionOnlyFeaturizer(DeBreuck2020Featurizer):
    def __init__(self):
        super().__init__()
        self.oxid_composition_featurizers = ()
        self.structure_featurizers = ()
        self.site_featurizers = ()

In [6]:

PRECOMPUTED_MODDATA = "./precomputed/expt_is_metal_benchmark_moddata.pkl.gz"

if os.path.isfile(PRECOMPUTED_MODDATA):
    data = MODData.load(PRECOMPUTED_MODDATA)
else:
    # Use a fresh copy of the dataset
    df = load_dataset("matbench_expt_is_metal")
    #df["composition"] = df["composition"].map(Composition)
    df["structure"] = df["composition"].map(Composition)
    
    data = MODData(
        materials=df["structure"].tolist(), 
        targets=df["is_metal"].tolist(), 
        target_names=["is_metal"],
        featurizer=CompositionOnlyFeaturizer(),
        num_classes = {'is_metal':2}
    )
    data.featurize()
    # As this is a small data/feature set, order all features 
    data.feature_selection(n=-1)
    data.save(PRECOMPUTED_MODDATA)

2023-02-07 13:47:26,408 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x1222f4070> object, created with modnet version 0.1.12


In [7]:

best_settings = {
    "increase_bs":False,
    "num_neurons": [[128], [32], [32], [16]],
    "n_feat": 120,
    "lr": 0.005, #0.005
    "epochs": 100,
    "verbose": 0,
    "act": "elu",
    "batch_size": 64,
    "num_classes": {'is_metal':2},
    "loss": "categorical_crossentropy",
    #"xscale": "standard",
}

In [8]:
data.split

<bound method MODData.split of <modnet.preprocessing.MODData object at 0x1222f4070>>

In [9]:
model = MODNetModel([[["is_metal"]]], {"is_metal": 1}, num_neurons= best_settings["num_neurons"],
            num_classes=best_settings.get("num_classes"),
            act=best_settings.get("act"),
            out_act=best_settings.get("out_act", "linear"),
            n_feat=best_settings["n_feat"],)

In [10]:
model.fit(data, epochs=best_settings["epochs"], batch_size=best_settings["batch_size"], 
verbose=best_settings["verbose"], lr=best_settings["lr"], loss=best_settings["loss"])

/Users/kevinmaikjablonka/miniconda3/envs/gptchem/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2023-02-07 13:47:27.021826: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [11]:
predictions = model.predict(data)

154/154 [==============================] - 0s 525us/step


In [12]:
predictions

,is_metal
id0,1
id1,1
id2,0
id3,0
id4,1
...,...
id4916,0
id4917,1
id4918,1
id4919,1
